In [1]:
import pandas as pd
import numpy as np

# Creating mock data

In [2]:
dates = pd.date_range('1/1/2000', periods=2000)
mock_frame = pd.DataFrame(index=dates)

In [3]:
mock_frame["close"] = np.random.rand(len(mock_frame), 1)

In [4]:
mock_frame["random_forest"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["svc"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["logreg"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))
mock_frame["gauss_nb"] = np.random.choice(["buy", "sell", "hold"], len(mock_frame))

In [5]:
mock_frame

,close,random_forest,svc,logreg,gauss_nb
2000-01-01,0.181182,hold,sell,hold,sell
2000-01-02,0.564426,buy,sell,buy,hold
2000-01-03,0.577285,hold,sell,buy,sell
2000-01-04,0.457348,hold,sell,sell,hold
2000-01-05,0.467620,sell,sell,sell,sell
...,...,...,...,...,...
2005-06-18,0.974986,buy,hold,buy,hold
2005-06-19,0.775248,sell,buy,hold,buy
2005-06-20,0.424707,hold,hold,sell,hold
2005-06-21,0.157685,buy,buy,buy,hold


In [6]:
mock_frame

,close,random_forest,svc,logreg,gauss_nb
2000-01-01,0.181182,hold,sell,hold,sell
2000-01-02,0.564426,buy,sell,buy,hold
2000-01-03,0.577285,hold,sell,buy,sell
2000-01-04,0.457348,hold,sell,sell,hold
2000-01-05,0.467620,sell,sell,sell,sell
...,...,...,...,...,...
2005-06-18,0.974986,buy,hold,buy,hold
2005-06-19,0.775248,sell,buy,hold,buy
2005-06-20,0.424707,hold,hold,sell,hold
2005-06-21,0.157685,buy,buy,buy,hold


In [7]:
res = pd.DataFrame(index=mock_frame.index, columns=mock_frame.columns[1:])

In [8]:
res = res.rename(columns={"random_forest": "random_forest_profit", "svc":"svc_profit", "logreg":"logreg_profit", "gauss_nb": "gauss_nb_profit"})

In [9]:
res

,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2000-01-01,NaN,NaN,NaN,NaN
2000-01-02,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN
...,...,...,...,...
2005-06-18,NaN,NaN,NaN,NaN
2005-06-19,NaN,NaN,NaN,NaN
2005-06-20,NaN,NaN,NaN,NaN
2005-06-21,NaN,NaN,NaN,NaN


## Profit calculations

Buy is only available while **not** holding shares.

Sell is only available if you **are** holding shares.

If **neither** buy or sell is available, the decision is hold.

Currently only 1 share is bought, and only 1 share is sold per occurence.

The share value is based on the current closing price.

In [10]:
# LOOP THROUGH COLUMNS
for col in mock_frame.columns[1:]:
    current_profit = 0
    holding = False
    holding_value = 0
    resulting_col = col + "_profit"
    
    # LOOP THROUGH EACH ROW
    for i in range(len(mock_frame)):

        if((holding == False) & (mock_frame[col].iloc[i] == "buy")): # ON BUY
            
            # SET HOLDING VALUE TO CURRENT CLOSE PRICE
            holding_value = mock_frame["close"].iloc[i]
            
             # ADD NEW PROFIT TO DATAFRAME
            res[resulting_col].iloc[i] = current_profit
            
            # UPDATE HOLDING VALUES
            holding = True
            
        elif((holding == True) & (mock_frame[col].iloc[i] == "sell")): # ON SELL
            
            # SET NEW PROFIT VALUE TO CURRENT SELL VALUE - PREVIOUS BUY VALUE
            current_profit += mock_frame["close"].iloc[i] - holding_value
            
            # ADD NEW PROFIT TO DATAFRAME
            res[resulting_col].iloc[i] = current_profit
            
             # UPDATE HOLDING VALUES
            holding_value = 0
            holding = False
            
        else: # ON HOLD
            
            # ADD NEW PROFIT TO DATAFRAME
            res[resulting_col].iloc[i] = current_profit

In [11]:
res

,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2000-01-01,0,0,0,0
2000-01-02,0,0,0,0
2000-01-03,0,0,0,0
2000-01-04,0,0,-0.107078,0
2000-01-05,-0.0968057,0,-0.107078,0
...,...,...,...,...
2005-06-18,2.56726,0.625488,-16.1626,-0.946463
2005-06-19,3.01777,0.625488,-16.1626,-0.946463
2005-06-20,3.01777,0.625488,-16.7129,-0.946463
2005-06-21,3.01777,0.625488,-16.7129,-0.946463


In [15]:
final = pd.concat([mock_frame, res], axis=1)

In [17]:
final[:50]

,close,random_forest,svc,logreg,gauss_nb,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2000-01-01,0.181182,hold,sell,hold,sell,0,0,0,0
2000-01-02,0.564426,buy,sell,buy,hold,0,0,0,0
2000-01-03,0.577285,hold,sell,buy,sell,0,0,0,0
2000-01-04,0.457348,hold,sell,sell,hold,0,0,-0.107078,0
2000-01-05,0.467620,sell,sell,sell,sell,-0.0968057,0,-0.107078,0
2000-01-06,0.668474,hold,sell,sell,sell,-0.0968057,0,-0.107078,0
2000-01-07,0.624438,sell,hold,hold,buy,-0.0968057,0,-0.107078,0
2000-01-08,0.033487,sell,hold,sell,sell,-0.0968057,0,-0.107078,-0.590951
2000-01-09,0.805918,hold,hold,buy,hold,-0.0968057,0,-0.107078,-0.590951
2000-01-10,0.174949,buy,buy,hold,sell,-0.0968057,0,-0.107078,-0.590951


In [21]:
final[-50:]

,close,random_forest,svc,logreg,gauss_nb,random_forest_profit,svc_profit,logreg_profit,gauss_nb_profit
2005-05-04,0.954163,sell,buy,hold,sell,2.71564,1.00253,-13.1173,-2.79695
2005-05-05,0.345336,hold,hold,buy,hold,2.71564,1.00253,-13.1173,-2.79695
2005-05-06,0.460287,sell,buy,hold,hold,2.71564,1.00253,-13.1173,-2.79695
2005-05-07,0.213446,sell,hold,buy,buy,2.71564,1.00253,-13.1173,-2.79695
2005-05-08,0.622689,sell,hold,buy,hold,2.71564,1.00253,-13.1173,-2.79695
2005-05-09,0.860908,sell,sell,buy,sell,2.71564,0.90928,-13.1173,-2.14949
2005-05-10,0.035379,hold,buy,sell,sell,2.71564,0.90928,-13.4272,-2.14949
2005-05-11,0.263162,buy,hold,sell,hold,2.71564,0.90928,-13.4272,-2.14949
2005-05-12,0.361687,hold,buy,hold,hold,2.71564,0.90928,-13.4272,-2.14949
2005-05-13,0.681826,buy,buy,buy,hold,2.71564,0.90928,-13.4272,-2.14949
